# I. Define the Problem
For this small project I am going to analyze characteristics of baseball players. We want to build a model to predict the Position of a player based on these characteristics using features like age, height, weight, throwing and batting arm.

# II. Collect the Data
First step is to load our source data into our environment in order to begin our analysis. To do this we will need to collect some data from [Baseball Reference's](https://www.baseball-reference.com) website by leveraging both the **Requests** and **BeautifulSoup** libraries. These two libraries help us easily access website data and scrape the datapoints we need for our analysis. 

In order for to know what to scrape we need to inspect the webpage's source code to narrow down what elements need captured from the HTML and what page elements do not. Here we are going to pull all the data from an HTML table with the id of 'appearances'. This html element holds all the data we need for our analysis. Since the element is an HTML table, we know the data is already structured, so we will store our HTML table data in a dataframe by leveraging the **Pandas** library. **Pandas** is a handy open-source data analysis library, it is particularly useful for structured data operations and manipulations.

In [43]:
# Import Python Libraries
import pandas as pd             # for structured data operations and manipulations
import numpy as np              # for scientific computing 
import requests                 # for accessing the web
from bs4 import BeautifulSoup   # for scraping information from web pages

There is a chance that we might need to more data from this data source later in our analysis, so let's take some time now to create a reuseable function to make this task easier. The below function will take two parameters, 'teams' and 'years'. Both parameters will be of List datatype. This will allow us to dynamically build out dataframe to increase the amount of data being collected by team and time (years). Since this function can be reused, its a good idea to add some basic logging and error-handling so if the results of a function call are unexpected, then we have a starting point to start digging for the problem and debug.

In [44]:
def get_baseball_roster_stats(list_of_team_city_abbrv=['PIT'], list_of_years=[2014]):
    """ Returns *list_of_teams* for *list_of_years* player roster data.
    
    Usage::
        >>> df_baseball_rosters = get_baseball_roster_stats(['PIT', 'ATL'], [2014,2015])
        
    :param list_of_team_city_abbrv: A list of city abbreviations for baseball teams. Default = 'PIT'
    :param list_of_years: A list of integers representing years. Default = 2014
    :rtype: A Pandas dataframe
    """
    # Create base url for data source to build out dynamically: url_roster_base
    url_roster_base = 'https://www.baseball-reference.com/teams/'
    
    # Initialize return dataframe to hold rosters data: df_rosters
    df_rosters = pd.DataFrame()
    
    for team in list_of_team_city_abbrv:
        # Add team city abbreviation to base URL: url_roster_team
        url_roster_team = url_roster_base + team + '/'
        
        # !!![Debug]!!! - Check the dynamic URL build process, added team
        #print('Add team to URL: ' + url_roster_team)
        
        for year in list_of_years:
            # Add year and url suffix to dynamic URL: url_roster_year
            url_roster_year = url_roster_team + str(year) + '-roster.shtml'
            
            # !!![Debug]!!! - Check the dynamic URL build process, added year
            #print('Team Year URL: ' + url_roster_year)
            
            try:
                # Request GET for dynamically built URL for team and year: res
                res = requests.get(url_roster_year)
                
                # Get the HTML/XML source code for the webpage: soup
                soup = BeautifulSoup(res.content,'lxml')
                
                # Search HTML elements to get targeted table's content: table
                table = soup.find('table', id='appearances')
                
                # Read table contents into a Pandas dataframe: df
                df = pd.read_html(str(table))[0]
                df['Year'] = year
                df['Team'] = team
                
                # Add year-team created dataframe to returning datafame (df_rosters)
                df_rosters = df_rosters.append(df)

                # Logging
                print('Successfully loaded data for {0}''s {1} team.'.format(team, year))
                
            except:
                # If an error occurs with the HTML scraping or Dataframe load, raise this Error Message.
                print('Webscraping Error: Check URL and HTML Table for Changes/Issues.')
            
            # Reset URL to build out for next year for current team
            url_roster_year = ''
            
    return df_rosters

Now that we have created this handy function to help with our **data collection** let's get started by selecting one team to focus on, but to grab enough data to start our analysis lets grab multiple years of data.

In [45]:
df_all_rosters = get_baseball_roster_stats(['PIT'], range(2000,2019))

Successfully loaded data for PITs 2000 team.
Successfully loaded data for PITs 2001 team.
Successfully loaded data for PITs 2002 team.
Successfully loaded data for PITs 2003 team.
Successfully loaded data for PITs 2004 team.
Successfully loaded data for PITs 2005 team.
Successfully loaded data for PITs 2006 team.
Successfully loaded data for PITs 2007 team.
Successfully loaded data for PITs 2008 team.
Successfully loaded data for PITs 2009 team.
Successfully loaded data for PITs 2010 team.
Successfully loaded data for PITs 2011 team.
Successfully loaded data for PITs 2012 team.
Successfully loaded data for PITs 2013 team.
Successfully loaded data for PITs 2014 team.
Successfully loaded data for PITs 2015 team.
Successfully loaded data for PITs 2016 team.
Successfully loaded data for PITs 2017 team.
Successfully loaded data for PITs 2018 team.


# III. Explore the Data
Now we have our data imported into our environment, we need to start exploring our data to understand what we are working with more. In this stage we will plot data, look for anomalies and identify any useful patterns

## Exploratory Data Analysis - Quantitative
A good starting point in Exploratory Data Analysis (EDA) exercise is to look at high-level/summary data, as well as the metadata of our dataset to help us understand the data a bit better.

We'll start by checking out the first 10 rows of the DataFrame to get a quick look at the dataset.

In [46]:
print(df_all_rosters.head(10))

                 Name Age Unnamed: 2  B  T     Ht   Wt           DoB  Yrs  \
0      Jimmy Anderson  24      us US  L  L  6' 1"  195  Jan 22, 1976    2   
1      Bronson Arroyo  23      us US  R  R  6' 3"  185  Feb 24, 1977  1st   
2          Bruce Aven  28      us US  R  R  5' 9"  180   Mar 4, 1972    3   
3       Mike Benjamin  34      us US  R  R  6' 3"  195  Nov 22, 1965   12   
4         Kris Benson  25      us US  R  R  6' 4"  190   Nov 7, 1974    2   
5        Adrian Brown  26      us US  B  R  6' 0"  175   Feb 7, 1974    4   
6          Emil Brown  25      us US  R  R  6' 2"  195  Dec 29, 1974    4   
7  Jason Christiansen  30      us US  R  L  6' 5"  230  Sep 21, 1969    6   
8         Brad Clontz  29      us US  R  R  6' 1"  180  Apr 25, 1971    6   
9         Wil Cordero  28      pr PR  R  R  6' 2"  185   Oct 3, 1971    9   

     G ...   RF  OF DH  PH PR   WAR      Salary Unnamed: 28  Year Team  
0   27 ...    0   0  0   0  0   0.7    $215,000         NaN  2000  PIT  
1   21

By simply previewing our dataframe we can derive some insight immediately. For instance, we have a very diverse set of data, some numerical, some date related, some categorical, etc. One way to gain a better understanding of the metadata is to review a more concise summary of the DataFrame. The **info()** method in the **Pandas** library allows us to do just that by printing information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage.

In [47]:
# View information about the DataFrame metadata
df_all_rosters.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 912 entries, 0 to 48
Data columns (total 31 columns):
Name           912 non-null object
Age            912 non-null object
Unnamed: 2     893 non-null object
B              912 non-null object
T              912 non-null object
Ht             912 non-null object
Wt             912 non-null object
DoB            912 non-null object
Yrs            912 non-null object
G              912 non-null object
GS             912 non-null object
Batting        912 non-null object
Defense        912 non-null object
P              912 non-null object
C              912 non-null object
1B             912 non-null object
2B             912 non-null object
3B             912 non-null object
SS             912 non-null object
LF             912 non-null object
CF             912 non-null object
RF             912 non-null object
OF             912 non-null object
DH             912 non-null object
PH             912 non-null object
PR             912 no

We can dervive additional information about our dataframe by reviewing the metadata above. For instance, we can see that there are a total of 31 columns and 912 rows or samples in our dataset. However, if we look at the non-null values for each column we see that the 'Unnamed: 2', 'Salary', 'Unnamed: 28' columns are missing data. This means we need to investigate these columns to determine if **data cleansing** is needed. 

## Cleanse the Data
It is common to complete some data cleansing or wrangling processes while we explore and learn more about our data. Data cleansing is important to make sure our data is in a format and structure that can be easily consumed by our analysis and models. This might include task such as checking for proper data types and converting data to correct data types/formats when needed. This can also involve stripping characters from the strings. There might also be some missing values in the dataset that need to be handled by adding or deleting some values.

So let's again preview the data from above, but we will narrow it to just these rows in question and see if there is any data cleansing needed to add more value to this dataset.

In [48]:
print(df_all_rosters[['Unnamed: 2', 'Salary', 'Unnamed: 28']].head(10))

  Unnamed: 2      Salary Unnamed: 28
0      us US    $215,000         NaN
1      us US         NaN         NaN
2      us US    $320,000         NaN
3      us US    $700,000         NaN
4      us US    $300,000         NaN
5      us US    $285,000         NaN
6      us US         NaN         NaN
7      us US  $1,000,000         NaN
8      us US         NaN         NaN
9      pr PR  $2,166,667         NaN


Since two of these columns have ambigious or unclear labels, its hard to determine if these columns are valueable and if so, can we clean or transform the data to provide more insight. So, let's investigate each of these columns seperately, starting with 'Unnamed: 28'. The first question to answer is, what values are in the column that are non-null. This will give us information on what this column represents.

In [49]:
# Get unique values in column 'Unnamed: 28'
print(df_all_rosters[df_all_rosters['Unnamed: 28'].notnull()]['Unnamed: 28'].unique())

['All-Star']


It appears that this column indicates if a player was categorized as an 'All-Star' player for the season or not. It can be assumed that the NaN value means the player was not an All-Star. So let's clean this column up by replacing NaN with 'Regular'. But first, let's create a copy of the original dataframe to use going forward that will be cleansed and transformed, allowing us to still have the original 'raw' dataframe stored incase we need it.

In [50]:
# Create copy of dataframe to maintain original: df_all_rosters_clean
df_all_rosters_clean = df_all_rosters.copy()

# Replace all NaN values in 'Unnamed: 28' column with 'Regular'
df_all_rosters_clean['Unnamed: 28'].fillna('Regular', inplace=True)

# Check
print(df_all_rosters_clean[df_all_rosters_clean['Unnamed: 28'].notnull()]['Unnamed: 28'].unique())

['Regular' 'All-Star']


Now let's tackle our other ambigious column 'Unnamed: 2', this one is a little more clear, it looks like it might be the country the player is from, but let's take deeper look.

In [51]:
# Get unique values in column 'Unnamed: 2'
print(df_all_rosters_clean[df_all_rosters_clean['Unnamed: 2'].notnull()]['Unnamed: 2'].unique())

['us US' 'pr PR' 'mx MX' 'do DO' 've VE' 'ca CA' 'cw CW' 'jp JP' 'cu CU'
 'kr KR' 'nl NL' 'co CO' 'lt LT' 'za ZA']


After reviewing the unique values in our 'Unnamed: 2' column and going back to the [data source itself](https://www.baseball-reference.com/teams/PIT/2018-roster.shtml) to evaluate, we can conclude this column is the player's home country, however the table column cell has the country's abbreviation in caps next to an image of it's flag. We will remove the lower-case values and keep the upper-case values for this columns.

In [52]:
# Clean duplicate country code, keep only uppercase value
df_all_rosters_clean['Unnamed: 2'] = df_all_rosters_clean['Unnamed: 2'].apply(lambda x: x.split(' ')[1] if isinstance(x, str) else x)

# Check unique values in column 'Unnamed: 2'
print(df_all_rosters_clean[df_all_rosters_clean['Unnamed: 2'].notnull()]['Unnamed: 2'].unique())

['US' 'PR' 'MX' 'DO' 'VE' 'CA' 'CW' 'JP' 'CU' 'KR' 'NL' 'CO' 'LT' 'ZA']


I think for the time being, leaving the NaN value in the 'Unnamed: 2' column makes sense, replacing with a dummy value like 'Unknown' adds no additional value to our analysis as of now. For the time being we will not address the Salary as well. Let's check how our dataset looks now after our first phase of data cleansing. But this time, lets look at the first 15 columns first, and then the remaining columns to see if we can identify other columns that might need **data cleansing**.

In [53]:
print(df_all_rosters_clean.iloc[:,0:15].head(10))
print(df_all_rosters_clean.iloc[:,0:15].tail(10))

                 Name Age Unnamed: 2  B  T     Ht   Wt           DoB  Yrs  \
0      Jimmy Anderson  24         US  L  L  6' 1"  195  Jan 22, 1976    2   
1      Bronson Arroyo  23         US  R  R  6' 3"  185  Feb 24, 1977  1st   
2          Bruce Aven  28         US  R  R  5' 9"  180   Mar 4, 1972    3   
3       Mike Benjamin  34         US  R  R  6' 3"  195  Nov 22, 1965   12   
4         Kris Benson  25         US  R  R  6' 4"  190   Nov 7, 1974    2   
5        Adrian Brown  26         US  B  R  6' 0"  175   Feb 7, 1974    4   
6          Emil Brown  25         US  R  R  6' 2"  195  Dec 29, 1974    4   
7  Jason Christiansen  30         US  R  L  6' 5"  230  Sep 21, 1969    6   
8         Brad Clontz  29         US  R  R  6' 1"  180  Apr 25, 1971    6   
9         Wil Cordero  28         PR  R  R  6' 2"  185   Oct 3, 1971    9   

     G  GS Batting Defense   P  C  
0   27  26      27      27  27  0  
1   21  12      21      20  20  0  
2   72  26      72      41   0  0  
3   93  

It would appear that we have potentially records we did not intend to scrape from the data source to our dataframe. The last record of the dataframe is a repeat of the columns names. Let's remove any rows where the value of the 'Name' column is 'Name'. That should clean that up.

In [54]:
row_count_prior = df_all_rosters_clean.shape[0]

df_all_rosters_clean = df_all_rosters_clean[df_all_rosters_clean.Name != 'Name']

row_count_after = df_all_rosters_clean.shape[0]
print('A Total of {} rows were removed from the dataframe.'.format(row_count_prior-row_count_after))

A Total of 19 rows were removed from the dataframe.


The only column that appears to need addition data cleansing or transformation is the 'Ht' column that represents the player's height in the format of Feet' Inches", therefore this column was identified as an object or string column. Let's convert this column to represent the player's height in inches so this then is easier to handle in our analysis as it will then be numberic.

In [55]:
# Feet to Inches convert function
fn_height_convert = lambda x: int(x.split("'")[0])*12 + int(x.split("'")[1].replace('"', ''))

# Clean Height column by converting string representation of feet to numeric representation of inches
df_all_rosters_clean['Ht'] = df_all_rosters_clean['Ht'].apply(fn_height_convert)

# Test
df_all_rosters_clean.iloc[:,0:15].head(10)

,Name,Age,Unnamed: 2,B,T,Ht,Wt,DoB,Yrs,G,GS,Batting,Defense,P,C
0,Jimmy Anderson,24,US,L,L,73,195,"Jan 22, 1976",2,27,26,27,27,27,0
1,Bronson Arroyo,23,US,R,R,75,185,"Feb 24, 1977",1st,21,12,21,20,20,0
2,Bruce Aven,28,US,R,R,69,180,"Mar 4, 1972",3,72,26,72,41,0,0
3,Mike Benjamin,34,US,R,R,75,195,"Nov 22, 1965",12,93,54,93,85,0,0
4,Kris Benson,25,US,R,R,76,190,"Nov 7, 1974",2,32,32,30,32,32,0
5,Adrian Brown,26,US,B,R,72,175,"Feb 7, 1974",4,104,63,104,92,0,0
6,Emil Brown,25,US,R,R,74,195,"Dec 29, 1974",4,50,26,50,38,0,0
7,Jason Christiansen,30,US,R,L,77,230,"Sep 21, 1969",6,44,0,41,44,44,0
8,Brad Clontz,29,US,R,R,73,180,"Apr 25, 1971",6,5,0,5,5,5,0
9,Wil Cordero,28,PR,R,R,74,185,"Oct 3, 1971",9,89,84,89,85,0,0


While we are in a data cleansing mood, lets convert some of these object features to type 'category' where it makes sense. This is a good practice as the Category data type uses less memory which will improve performance for some operations, such as the **groupby()** operation. We should also make sure all the other columns are of optimal data type for our analysis.

In [56]:
# Convert necessary object columns to numeric
df_all_rosters_clean[['Age', 'Wt', 'G', 'GS', 'Batting', 'Defense', 'P', 'C', '1B', '2B', '3B', 'SS', 'LF',
       'CF', 'RF', 'OF', 'DH', 'PH', 'PR', 'WAR']] = df_all_rosters_clean[['Age', 'Wt', 'G', 'GS', 'Batting', 'Defense', 'P', 'C', '1B', '2B', '3B', 'SS', 'LF',
       'CF', 'RF', 'OF', 'DH', 'PH', 'PR', 'WAR']].apply(pd.to_numeric)

# Convert necessary object columns to category
df_all_rosters_clean[['Unnamed: 2', 'B', 'T', 'Team']] = df_all_rosters_clean[['Unnamed: 2', 'B', 'T', 'Team']].apply(pd.Categorical)

#
df_all_rosters_clean['Salary'] = df_all_rosters_clean['Salary'].replace('[\$,]', '', regex=True).astype(float)

#d.to_numeric(df_all_rosters_clean['Age'])
df_all_rosters_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893 entries, 0 to 47
Data columns (total 31 columns):
Name           893 non-null object
Age            893 non-null int64
Unnamed: 2     893 non-null category
B              893 non-null category
T              893 non-null category
Ht             893 non-null int64
Wt             893 non-null int64
DoB            893 non-null object
Yrs            893 non-null object
G              893 non-null int64
GS             893 non-null int64
Batting        893 non-null int64
Defense        893 non-null int64
P              893 non-null int64
C              893 non-null int64
1B             893 non-null int64
2B             893 non-null int64
3B             893 non-null int64
SS             893 non-null int64
LF             893 non-null int64
CF             893 non-null int64
RF             893 non-null int64
OF             893 non-null int64
DH             893 non-null int64
PH             893 non-null int64
PR             893 non-null int64


Next step is to clean up our column names, to provide more informative, descriptive names to help us with our analysis.

In [57]:
df_all_rosters_clean.iloc[:,15:].head(10)
df_all_rosters_clean.iloc[:,15:].tail(10)

,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR,WAR,Salary,Unnamed: 28,Year,Team
38,0,0,0,0,0,0,0,0,0,0,0,1.7,NaN,Regular,2018,PIT
39,2,26,1,16,9,6,4,18,0,12,3,-0.4,5750000.0,Regular,2018,PIT
40,0,0,0,0,0,0,0,0,0,0,0,-0.1,NaN,Regular,2018,PIT
41,0,0,0,0,0,0,0,0,0,0,0,-0.3,NaN,Regular,2018,PIT
42,0,0,0,0,0,0,0,0,0,0,0,1.1,556500.0,Regular,2018,PIT
43,0,0,0,0,0,0,0,0,0,0,0,-0.4,570500.0,Regular,2018,PIT
44,0,0,0,0,0,0,0,0,0,3,0,-0.1,NaN,Regular,2018,PIT
45,0,0,0,0,0,0,0,0,0,0,0,4.4,NaN,Regular,2018,PIT
46,0,0,0,0,0,0,0,0,0,0,0,1.5,3000000.0,All-Star,2018,PIT
47,0,0,0,0,0,0,0,0,0,0,0,3.8,569500.0,Regular,2018,PIT


Another good practice is if a date feature exists, make sure its datatype is datetime related. This will allow us to take full advantage of any data and time-series operations during our analysis.

In [58]:
from dateutil import parser

df_all_rosters_clean['DoB'] = df_all_rosters_clean['DoB'].apply(lambda x: parser.parse(x))
df_all_rosters_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893 entries, 0 to 47
Data columns (total 31 columns):
Name           893 non-null object
Age            893 non-null int64
Unnamed: 2     893 non-null category
B              893 non-null category
T              893 non-null category
Ht             893 non-null int64
Wt             893 non-null int64
DoB            893 non-null datetime64[ns]
Yrs            893 non-null object
G              893 non-null int64
GS             893 non-null int64
Batting        893 non-null int64
Defense        893 non-null int64
P              893 non-null int64
C              893 non-null int64
1B             893 non-null int64
2B             893 non-null int64
3B             893 non-null int64
SS             893 non-null int64
LF             893 non-null int64
CF             893 non-null int64
RF             893 non-null int64
OF             893 non-null int64
DH             893 non-null int64
PH             893 non-null int64
PR             893 non-nul

In [59]:
df_all_rosters_clean.columns

Index(['Name', 'Age', 'Unnamed: 2', 'B', 'T', 'Ht', 'Wt', 'DoB', 'Yrs', 'G',
       'GS', 'Batting', 'Defense', 'P', 'C', '1B', '2B', '3B', 'SS', 'LF',
       'CF', 'RF', 'OF', 'DH', 'PH', 'PR', 'WAR', 'Salary', 'Unnamed: 28',
       'Year', 'Team'],
      dtype='object')

In [60]:
roster_columns = ['player_name','player_age','country','batting_side','throwing_side','height','weight','dob','yrs_exp',
                  'num_games_played', 'num_games_started', 'num_games_batorder', 'num_games_def', 'num_games_pitcher',
                  'num_games_catcher', 'num_games_1b', 'num_games_2b', 'num_games_3b', 'num_games_ss', 'num_games_lft_fld',
                  'num_games_c_fld','num_games_rgt_fld', 'num_games_out_fld', 'num_games_desig_hit', 'num_games_pin_hit',
                  'num_games_pin_run','wins_above_replace', 'salary', 'allstar_status', 'roster_yr', 'city'
                 ]

df_all_rosters_clean.columns = roster_columns
print(df_all_rosters_clean.columns)

Index(['player_name', 'player_age', 'country', 'batting_side', 'throwing_side',
       'height', 'weight', 'dob', 'yrs_exp', 'num_games_played',
       'num_games_started', 'num_games_batorder', 'num_games_def',
       'num_games_pitcher', 'num_games_catcher', 'num_games_1b',
       'num_games_2b', 'num_games_3b', 'num_games_ss', 'num_games_lft_fld',
       'num_games_c_fld', 'num_games_rgt_fld', 'num_games_out_fld',
       'num_games_desig_hit', 'num_games_pin_hit', 'num_games_pin_run',
       'wins_above_replace', 'salary', 'allstar_status', 'roster_yr', 'city'],
      dtype='object')


In [61]:
df_all_rosters_clean.drop(['yrs_exp','dob'], axis=1, inplace=True)

df_all_rosters_clean.iloc[:,:15].head(10)

,player_name,player_age,country,batting_side,throwing_side,height,weight,num_games_played,num_games_started,num_games_batorder,num_games_def,num_games_pitcher,num_games_catcher,num_games_1b,num_games_2b
0,Jimmy Anderson,24,US,L,L,73,195,27,26,27,27,27,0,0,0
1,Bronson Arroyo,23,US,R,R,75,185,21,12,21,20,20,0,0,0
2,Bruce Aven,28,US,R,R,69,180,72,26,72,41,0,0,0,0
3,Mike Benjamin,34,US,R,R,75,195,93,54,93,85,0,0,1,27
4,Kris Benson,25,US,R,R,76,190,32,32,30,32,32,0,0,0
5,Adrian Brown,26,US,B,R,72,175,104,63,104,92,0,0,0,0
6,Emil Brown,25,US,R,R,74,195,50,26,50,38,0,0,0,0
7,Jason Christiansen,30,US,R,L,77,230,44,0,41,44,44,0,0,0
8,Brad Clontz,29,US,R,R,73,180,5,0,5,5,5,0,0,0
9,Wil Cordero,28,PR,R,R,74,185,89,84,89,85,0,0,0,0


In [62]:
df_all_rosters_clean.iloc[:,15:].head(10)

,num_games_3b,num_games_ss,num_games_lft_fld,num_games_c_fld,num_games_rgt_fld,num_games_out_fld,num_games_desig_hit,num_games_pin_hit,num_games_pin_run,wins_above_replace,salary,allstar_status,roster_yr,city
0,0,0,0,0,0,0,0,0,0,0.7,215000.0,Regular,2000,PIT
1,0,0,0,0,0,0,0,1,1,-1.1,NaN,Regular,2000,PIT
2,0,0,17,8,20,45,0,35,0,-0.4,320000.0,Regular,2000,PIT
3,34,30,0,0,0,0,0,9,7,1.0,700000.0,Regular,2000,PIT
4,0,0,0,0,0,0,0,0,0,4.9,300000.0,Regular,2000,PIT
5,0,0,7,71,15,93,0,23,0,1.6,285000.0,Regular,2000,PIT
6,0,0,14,12,18,44,0,15,1,-0.5,NaN,Regular,2000,PIT
7,0,0,0,0,0,0,0,0,0,0.2,1000000.0,Regular,2000,PIT
8,0,0,0,0,0,0,0,0,0,0.1,NaN,Regular,2000,PIT
9,0,0,85,0,0,85,1,4,0,-0.3,2166667.0,Regular,2000,PIT


### Feature Selection
Since there are a large number (105) of features in this dataset this would take a really long time to train and test, as well as leading to potential overfitting. Next, we need to determine which features we think are likely to be important our target variable to help reduce the computational workload and improve performance. 

One way is review a more concise summary of the DataFrame. The **info()** method in the **Pandas** library allows us to do just that by printing information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage.

In [63]:
def get_pos_class(s):    
    if s['num_games_pitcher'] > 0:
        return 'Pitcher'
    elif s['num_games_catcher'] > 0:
        return 'Catcher'
    elif (s['num_games_1b'] > 0) | (s['num_games_2b'] > 0) | (s['num_games_3b'] > 0) | (s['num_games_ss'] > 0):  
        return 'Inner Fielder'
    elif (s['num_games_lft_fld']) | (s['num_games_c_fld']) | (s['num_games_rgt_fld']):
        return 'Out Fielder'
    else:
        return 'Unknown'

In [64]:
#df_all_rosters_fest = set_pos_class(df_all_rosters_clean)
df_all_rosters_clean['position_class'] = df_all_rosters_clean.apply(get_pos_class, axis=1)

df_all_rosters_clean['position_class'].head()

0          Pitcher
1          Pitcher
2      Out Fielder
3    Inner Fielder
4          Pitcher
Name: position_class, dtype: object

In [65]:
df_all_rosters_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893 entries, 0 to 47
Data columns (total 30 columns):
player_name            893 non-null object
player_age             893 non-null int64
country                893 non-null category
batting_side           893 non-null category
throwing_side          893 non-null category
height                 893 non-null int64
weight                 893 non-null int64
num_games_played       893 non-null int64
num_games_started      893 non-null int64
num_games_batorder     893 non-null int64
num_games_def          893 non-null int64
num_games_pitcher      893 non-null int64
num_games_catcher      893 non-null int64
num_games_1b           893 non-null int64
num_games_2b           893 non-null int64
num_games_3b           893 non-null int64
num_games_ss           893 non-null int64
num_games_lft_fld      893 non-null int64
num_games_c_fld        893 non-null int64
num_games_rgt_fld      893 non-null int64
num_games_out_fld      893 non-null int64
num_

Since we have both the Height and Weight of each players, we can also calculate a player's Body Mass Index (BMI), so let's that in our dataset.

In [66]:
df_all_rosters_clean['BMI'] = df_all_rosters_clean.apply(lambda x: (x['weight']*0.453592)/(x['height']* 0.0254)**2, axis=1)
print(df_all_rosters_clean['BMI'].head())

0    25.726864
1    23.123158
2    26.581059
3    24.373059
4    23.127270
Name: BMI, dtype: float64


### Analytic Approach
Now that we have a better idea of the data within the DataFrame we need to make three distinctions to choice the right path forward with our analysis.

1. Is this going to be supervised or unsupervised learning?   
We have a defined/labeled dependent or y variable, 'Position', allowing **Supervised Learning** to be performed. The computer to learn from our clearly labeled dataset.   


2. Is this a classification or regression problem?   
One way to identify the type of problem is to look at the y variable to see if it is discrete or continuous, and if its categorical or quantitative. Since 'Position' is discrete and categorical in nature, this is a **Classification** problem, specifically this is a **Multi-Class Classification** problem because each hard drive for a given day has either failed or not.


3. Is this a prediction or inference problem?   
The business use case is to create a model to be leverage to estimate a player's position, therefore this is **Prediction** problem. We will want the model to estimate a y ('Position') value, given a variety of features.   

## Exploratory Data Analysis - Quantitative
Now that we have loaded our dataset into a **Pandas** dataframe lets take a look at to understand our data better.

In [67]:
# Explore Dimensions of the Dataset

# Review the shape (rows, columns) of df_rosters
print(df_rosters.shape)

NameError: name 'df_rosters' is not defined

We can see that our dataset has 196 rows or observations and 9 columns or features. Let's take a quick look at the data of the top 10 rows.

In [ ]:
# Preview the first 10 rows/observations
df_rosters.head(10)

Now that we have a better understanding of the general shape of the data and have previewed the dataset, lets look at a statistical summary of out features. One useful method is the **Pandas** method **describe()**. This method provides some descriptive summary statisticals such as count, mean, minimum and maximum values for numeric features.

In [ ]:
df_rosters.describe()

Hmm.. We would expect summary statistics for more features, so let's take a look at the feature's metadata by leveraging the **Pandas** method **info()**.

In [ ]:
print(df_rosters.info())

Well there's our problem, some of the features are of object type that should be integer. So that means we need to do some **Data Cleansing**. Lets start by changing the data types of # and height to integer, by first exploring those two features again.

In [ ]:
print(df_rosters[['#','Height']].head())

By previewing these two features again, we can see that '#' seems like it will be a pretty simply converstion, so we will try the **.astype()** method to convert the '#' values to an integer.

In [ ]:
df_rosters_clean = df_rosters.copy()

try:
    df_rosters_clean['#'] = df_rosters_clean['#'].astype(int)
    print(df_rosters_clean['#'].head())
except:
    print('Value Error: Column Value must contain only numeric values in order to convert.')

Well we can see that there are some string literals mixed into the values within the '#' so we are unable to do a straight conversion with the **.astype()** method. We will need to use **Python Regular Expressions** to clean up this feature by removing all non-numeric characters from the value before converting to an integer. 

In [ ]:
import re
non_decimal = re.compile('\d')
df_rosters_clean['#'] = df_rosters_clean['#'].apply(lambda x: re.findall('\d+', x)[0]).astype(int)

print(df_rosters_clean['#'].head())

Now that we cleaned up the '#' feature, let's tackle the 'Height' feature, which will require a little bit more transformation than the '#' did. It appears to be in the 'Feet - Inches' format, but we want to convert to an integer value for inches or a decimal value for feet. For this Project we'll go with inches.

In [ ]:
# Feet to Inches convert function
fn_height_convert = lambda x: int(x.split('-')[0])*12 + int(x.split('-')[1])

# Clean Height column by converting string representation of feet to numeric representation of inches
df_rosters_clean['Height'] = df_rosters_clean['Height'].apply(fn_height_convert)

# Test
df_rosters_clean.head()

While we are in a data cleansing mood, lets convert some of these object features to type 'category' where it makes sense. This is a good practice as the Category data type uses less memory which will improve performance for some operations, such as the **groupby()** operation.

In [ ]:
# Convert appropriate object features to category - Best Practice.
df_rosters_clean['Position'] = df_rosters_clean['Position'].astype('category')
df_rosters_clean['Throws'] = df_rosters_clean['Throws'].astype('category')
df_rosters_clean['Bats'] = df_rosters_clean['Bats'].astype('category')
print(df_rosters_clean.info())

Another good practice is if a date feature exists, make sure its datatype is datetime related. This will allow us to take full advantage of any data and time-series operations during our analysis.

Now that we did some metadata clean-up for our dataset features, lets take one more look at the dataset metadata to make sure all our features are of appropriate type.

In [ ]:
print(df_rosters_clean.info()) 

Let's also take the opportunity to rename a few of the features to more descriptive names. This will help us in our analysis easily understand the data within the features.

In [ ]:
df_rosters_clean.rename(columns={'#': 'Number', 
                                 'Height': 'Height_Inc', 
                                 'Weight':'Weight_Lbs', 
                                 'Date Of Birth': 'DoB'}, inplace=True)

print(df_rosters_clean.columns)

The metadata looks good now, so lets generate our descriptive summary statistics again to gain a better understanding of our numeric data, by again, leveraging the **describe()** method.

In [ ]:
df_rosters_clean.describe()

Now let's take a look at the distribution of rows that belong to each Position by viewing an absolution count leveraging the **groupby** method in **Pandas**.

In [ ]:
print(df_rosters_clean.groupby('Position').size())

This is definitely useful information, however let's not forget that the dataset has multiple years of roster data, so the same player could be duplicated across years. So, lets add 'Year' to our **groupby()** method.

In [ ]:
print(df_rosters_clean.groupby(['Position', 'Year']).size())

## Feature Engineering
Now that we have a better understanding of our data, it has become obvious that some additional features could be generated based on the features we already have that could add to our analysis.


For instance, we have the Data of Birth (DoB) for each player, so we can calculate their age going into the Roster year.

In [ ]:
df_rosters_clean.head(10)

In [ ]:
df_rosters_clean['Age'] = df_rosters_clean.apply(lambda x: x['Year'] - x['DoB'].year, axis=1)

df_rosters_clean.head()

Since we have both the Height and Weight of each players, we can also calculate a player's Body Mass Index (BMI), so let's that in our dataset.

In [ ]:
df_rosters_clean['BMI'] = df_rosters_clean.apply(lambda x: (x['Weight_Lbs']*0.453592)/(x['Height_Inc']* 0.0254)**2, axis=1)
print(df_rosters_clean['BMI'].head())

In [ ]:
# Create helper function to determine BMI Classification per NIH
def get_bmi_class(bmi_value):
    """Returns the BMI classification based on the given bmi_value 
    parameter passed into the function."""
    
    # Initialize return variable
    bmi_class = 'Unknown'
    
    if bmi_value < 18.5:
        bmi_class = 'Underweight'
    elif bmi_value >= 18.5 and bmi_value <= 24.9:
        bmi_class = 'Normal'
    elif bmi_value >= 25.0 and bmi_value <= 29.9:
        bmi_class = 'Overweight'
    elif bmi_value >= 30.0 and bmi_value <= 39.9:
        bmi_class = 'Obesity'
    else:
        bmi_class = 'Extreme Obesity'
    
    return(bmi_class)

# Add new feature/column to label player's BMI classification per NIH: df_pirates_roster['BMI_class']
df_rosters_clean['BMI Classification'] = df_rosters_clean.apply(lambda row: get_bmi_class(row['BMI']), axis=1).astype('category')

print(df_rosters_clean['BMI Classification'].head())

## Exploratory Data Analysis - Visual
Now that we have a basic understanding of our data, to help us gain some futher insight into our data, we want to create a visual to show the distribution of some of our features. The **Matplotlib** package has some great visual graphs that can be easily implemented in **Python**, as well as the **Seaborn** visualization package for additional visual styling and plotting abilities.


To give a more clearer understanding of the distribution of the numeric features, let's create a box and whisker plots. 

In [ ]:
df_rosters_clean[['Height_Inc', 'Weight_Lbs', 'Age', 'BMI']].plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
plt.show()

In [ ]:
df_rosters_clean[['Height_Inc', 'Weight_Lbs', 'Age', 'BMI']].hist()
plt.show()

In [ ]:
from pandas.plotting import scatter_matrix

# scatter plot matrix
scatter_matrix(df_rosters_clean[['Height_Inc', 'Weight_Lbs', 'Age', 'BMI']])
plt.show()

In [ ]:
sns.heatmap(df_rosters_clean.corr(), square=True, cmap='RdYlGn')
plt.show()

# IV. Analyze the Data

In [ ]:
df_roster_data = df_rosters_clean[['Height_Inc', 'Weight_Lbs', 'Age', 'BMI']]
df_roster_target = df_rosters_clean.loc[:,'Position'].values.astype('object')

print('df_roster_data datatype = {0} and df_roster_target datatype = {1}'.format(type(df_roster_data), 
                                                                                 type(df_roster_target)))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_roster_data, 
                                                    df_roster_target, 
                                                    test_size=0.5, 
                                                    random_state = 42,
                                                    stratify=df_roster_target)

print('Training Data Shape: X - {0}, Y-{1} \n Testing Data Shape: X - {2}, Y - {3}'.format(X_train.shape,
                                                                                          y_train.shape,
                                                                                          X_test.shape,
                                                                                          y_test.shape))

## Model Selection

In [ ]:
from pprint import pprint 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

def get_classifier_accuracies_comparison(X_train, y_train, X_test, y_test):
    # Compare accuracy of default classification algorithms.
    # Define the classifiers
    classifiers = {'Logistic Regression':LogisticRegression(multi_class='multinomial', solver='newton-cg'), 
                   'SVM (Linear)':LinearSVC(dual=False), 
                   'SVM':SVC(gamma='auto'), 
                   'KNeighbors':KNeighborsClassifier(3)}

    classifiers_accuracy = {}

    
    # Fit the classifiers
    for label, c in classifiers.items():
        c.fit(X_train, y_train)
        c.predict
        accuracy_score = c.score(X_test, y_test)
        classifiers_accuracy.update( {label:accuracy_score} )
    
    return classifiers_accuracy

In [ ]:
classifes_compare = get_classifier_accuracies_comparison(X_train, y_train, X_test, y_test)

plt.bar(range(len(classifes_compare)), list(classifes_compare.values()), align='center')
plt.xticks(range(len(classifes_compare)), list(classifes_compare.keys()))
plt.ylabel('Classifier Accuracy Score (%)')
plt.xlabel('Classifier')
plt.title('Classification Algorithm Accuracy Comparison')
plt.show()
    
pprint(classifes_compare)

## Secondary Modeling

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
knn.predict(X_test)
print(knn.score(X_test, y_test))

In [ ]:
from sklearn.svm import SVC, LinearSVC

svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
svm.predict(X_test)
print(svm.score(X_test, y_test))